# Similarity, Neighbors, and Clustering



***

In [1]:
# Import the libraries we will be using
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.model_selection import cross_val_score
from collections import defaultdict

import sys
sys.path.append("..")
! git clone https://github.com/yizuc/datamining.git

from datamining.ds_utils.decision_surface import *

sns.set(font_scale=1.5)

Cloning into 'datamining'...
remote: Enumerating objects: 571, done.
remote: Counting objects: 100% (571/571), done.
remote: Compressing objects: 100% (330/330), done.
remote: Total 571 (delta 246), reused 527 (delta 221), pack-reused 0
Receiving objects: 100% (571/571), 165.92 MiB | 28.07 MiB/s, done.
Resolving deltas: 100% (246/246), done.


## Lights, camera, action!

You've been hired by Netflakes as a business analytics professional. Netflakes' primary business is its subscription-based streaming service which offers online streaming of a library of films and television programs, including those produced in-house (sounds familiar?). A major competitive advantage of Netflakes is its wide variety of films. You've been hired to understand what kind of movies people like. Hopefully, your analysis will inspire ideas for new movies! 

Let's start by taking a look at the data we have available.

In [ ]:
# Import the ratings dataset
df_ratings = pd.read_csv('/content/datamining/Module7_Similarity_Clusters/data/ratings.csv')
df_ratings.head()

In [ ]:
# Import movies dataset
df_movies = pd.read_csv('/content/datamining/Module7_Similarity_Clusters/data/movies.csv')
df_movies.head()

In [ ]:
print('The dataset contains: ', len(df_ratings), ' ratings of ', len(df_movies), ' movies.')

In [ ]:
# Replace name of (no genres listed)
df_movies.genres = df_movies.genres.replace("(no genres listed)", "NA")
# Counts per genre (films may appear in more than one genre)
pd.Series(df_movies.genres.str.cat(sep="|").split("|")).value_counts()

## Feature representation
How can we turn these ratings into a useful representation of user tastes (i.e., features)?


### Avg. Ratings
One way is to define each feature vector as movie ratings or average genre rating. Let's start with genre.

In [ ]:
def get_genre_ratings(ratings, movies, genres, mean=True):
    all_genre_df = pd.DataFrame()
    for genre in genres:        
        genre_movies = movies[movies['genres'].str.contains(genre)]
        relevant_ratings = ratings[ratings['movieId'].isin(genre_movies['movieId'])]
        if mean is True:
            single_genre_df = relevant_ratings.groupby(['userId'])['rating'].mean().round(2)
        else:
            single_genre_df = relevant_ratings.groupby(['userId'])['rating'].count()
        all_genre_df = pd.concat([all_genre_df, single_genre_df], axis=1)
    all_genre_df.columns = genres
    return all_genre_df

genres = ['Action', 'Comedy']
users_by_avg_genre_ratings = get_genre_ratings(df_ratings, df_movies, genres)
sns.lmplot(genres[0], genres[1], data=users_by_avg_genre_ratings, fit_reg=False, height=8)
plt.show()

All genre ratings seem to be positively correlated. What's going on?

### Number of ratings
Let's try as features the total numbers of movies of each genre that users have rated.

In [ ]:
genres = ['Action', 'Comedy']
users_by_total_genre_ratings = get_genre_ratings(df_ratings, df_movies, genres, False)
sns.lmplot(genres[0], genres[1], data=users_by_total_genre_ratings, fit_reg=False, height=8)
plt.show()

Still everything seems correlated, why?

### Percentage of ratings
What if we instead define the feature vector as the percentage of ratings belonging to each genre?

In [ ]:
def get_genre_shares(ratings, movies):
    all_genres = np.unique(df_movies.genres.str.cat(sep="|").split("|"))
    all_genre_df = pd.DataFrame()
    for genre in all_genres:
        genre_movies = movies[movies['genres'].str.contains(genre)]
        relevant_ratings = ratings[ratings['movieId'].isin(genre_movies['movieId'])]
        single_genre_df = relevant_ratings.loc[:, ['userId', 'rating']].groupby(['userId'])['rating'].count()
        all_genre_df = pd.concat([all_genre_df, single_genre_df], axis=1)
    # Get shares
    all_genre_df.columns = all_genres
    all_genre_df.fillna(0, inplace=True)
    all_genre_df = all_genre_df.div(ratings.groupby('userId').rating.count(), axis=0)
    return all_genre_df

genres = ['Action', 'Comedy']
users_by_rating_per = get_genre_shares(df_ratings, df_movies)
sns.lmplot(genres[0], genres[1], data=users_by_rating_per, fit_reg=False, height=8)
plt.show()

Now we see different correlations. Do you still see any issues with this representation?

## Similarity measures

Once we have objects described as data, we can compute the similarity between different objects. Each of the users is now described by their tastes for different genres. Let's keep the share of ratings representation for now.

In [ ]:
users_by_rating_per.head()

 So ... how can we tell if users have similar tastes? Generally, how can we compute similarity between users?  We've reduced this question to: how can we compute similarity between objects described as feature vectors.

There are many similarity measures.  Similarity is often cast as "closeness" in some space, as computed by a distance measure.  Often in data science, the terms similarity and distance are used interchangeably (a little strangely to the uninitiated). 

We'll use the library scipy.spatial.distance available [here](http://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

This library has functions to compute the distance between two numeric vectors. In particular, **pdist(X[, metric, p, w, V, VI])**	computes pairwise distances between the observations in n-dimensional space. _Metric parameter: The distance function can be ‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, ‘correlation’, ‘cosine’, ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘kulsinski’, ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’._

Here is a function that will compute the distance using as many metrics as you want:

In [11]:
def user_distance(users, userId, distance_measures, n):
    # We want a data frame to store the output
    # distance_measures is a list of the distance measures you want to compute (see below)
    # n is how many "most similar" to report
    distances = pd.DataFrame()
    # Find the location of the whiskey we are looking for
    user_location = np.where(users.index == userId)[0][0]
    # Go through all distance measures we care about
    for distance_measure in distance_measures:
        # Find all pairwise distances
        current_distances = distance.squareform(distance.pdist(users, distance_measure))
        # Get the closest n elements for the user we care about
        most_similar = np.argsort(current_distances[:, user_location])[0:n]
        # Append results (a new column to the dataframe with the name of the measure)
        distances[distance_measure] = list(zip(users.index[most_similar], current_distances[most_similar, user_location]))
    return distances

We can use the function `user_distance` to find the distance value of each user with respect to others. We'll start using Euclidean distance as our metric:

In [ ]:
user_distance(users_by_rating_per, 1, ['euclidean'], 6)

Now, let's use more meatrics.

In [ ]:
user_distance(users_by_rating_per, 1, ['euclidean', 'cityblock', 'cosine', 'correlation'], 10)

In [ ]:
users_by_rating_per.loc[[1, 63, 328, 212]].sort_values(by=1, axis=1, ascending=False)

## Clustering Methods

Similarity has many uses in data science.  One of the most commonly discussed is clustering: Can we find groups of users that are similar?

### Hierarchical Clustering

There are different ways to find similar groups.  One very common method is Hierarchical Clustering.

First let's look at a simple example to illustrate.  Given a set of records (A-F) with two features, we can visualize them on a 2 dimensional surface.  Clustering proceeds as follows.  First consider each point to be its own cluster.  Then, iteratively, group together the closest two clusters.  In the figure, circles were drawn in order of grouping.  The second diagram is a visualization of the hierarchy of groupings, called a "dendrogram."  You can clip it at any point, vertically, and get "the best" clustering for a certain number of groups.


<img src="https://github.com/yizuc/datamining/blob/master/Module7_Similarity_Clusters/images/cutting.png?raw=1" height=40% width=40%>

Let's examine the dendrogram(s) for our data, we'll be using the library: **scipy.cluster.hierarchy**

In [ ]:
sample = users_by_rating_per.sample(80, random_state=42)

# This function gets pairwise distances between observations in n-dimensional space (e.g., cosine, euclidean).
dists = distance.pdist(sample, metric="cosine")

# This scipy's function performs hierarchical/agglomerative clustering on the condensed distance matrix y.
# Method could be 'average' distance from points in cluster v to points in cluster w or the 'single' shortest distance
links = linkage(dists, method='single')

# Now we want to plot those 'links' using "dendrogram" function
plt.rcParams['figure.figsize'] = 32, 16

den = dendrogram(links)

plt.xlabel('Samples',fontsize=30)
plt.ylabel('Distance',fontsize=30)
plt.xticks(rotation=90,fontsize=20)
plt.yticks(fontsize=20)
plt.show()

It is common to cut dendrograms at a particular height and to then use the resulting clusters. However, watch out for two things when doing hierarchical clusters: (1) it does not scale well with large data sets and (2) it's very sensitive to outliers.

### KMeans

Another method for finding clusters is to use the KMeans algorithm to find a set of $k$ clusters. Here, unlike in hierarchical clustering, we define the number of clusters in advance. We'll use the library **sklearn.cluster**

In [ ]:
k_clusters = 2
kmeans = KMeans(n_clusters=k_clusters, random_state=42)
genres = ['Thriller', 'Mystery']
kmeans.fit(users_by_rating_per[genres])
users_by_rating_per['cluster'] = kmeans.predict(users_by_rating_per[genres])
sns.lmplot(genres[0], genres[1], data=users_by_rating_per, hue='cluster', fit_reg=False, height=8)
plt.show()

We are using euclidean distance to find these clusters. Do you have any concerns about this?

In [ ]:
df_normalized_genre = (users_by_rating_per - users_by_rating_per.mean())/users_by_rating_per.std()
k_clusters = 2
kmeans = KMeans(n_clusters=k_clusters, random_state=42)
genres = ['Thriller', 'Mystery']
kmeans.fit(df_normalized_genre[genres])
df_normalized_genre['cluster'] = kmeans.predict(df_normalized_genre[genres])
sns.lmplot(genres[0], genres[1], data=df_normalized_genre, hue='cluster', fit_reg=False, height=8)
plt.show()

KMeans is very sensitive to scale and will tend to cluster according to the features with greater variance.

What happens if we use all the genres to cluster users?

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
all_genres = df_normalized_genre.columns[df_normalized_genre.columns != 'cluster']
genres = ['Action', 'Comedy']
df_normalized_genre['cluster'] = kmeans.fit_predict(df_normalized_genre[all_genres]).astype(int)
sns.lmplot(genres[0], genres[1], data=df_normalized_genre, hue='cluster', fit_reg=False, height=8)
plt.show()

Then, how can we describe or name each cluster?

In [ ]:
centroids = df_normalized_genre.groupby('cluster').mean()
centroids

In [ ]:
data = defaultdict(list)
top = 3

clusters = sorted(df_normalized_genre.cluster.unique())
for cluster in clusters:
    tastes = centroids.loc[cluster].sort_values()
    for i, t in enumerate(tastes[:top].index):
        data["Dislike {0}".format(i + 1)].append("{0}: {1}".format(t, tastes[t].round(2)))
    for i, t in enumerate(tastes[-top:][::-1].index):
        data["Like {0}".format(i + 1)].append("{0}: {1}".format(t, tastes[t].round(2)))
    counts = df_normalized_genre[df_normalized_genre.cluster == cluster].shape[0]
    data["count"].append(counts)
    
cols = ['count'] + ["Like {0}".format(i+1) for i in range(top)] + ["Dislike {0}".format(i+1) for i in range(top)]
cluster_info = pd.DataFrame(data)[cols].transpose()
cluster_info.columns = ["cluster {0}".format(i) for i in clusters]
cluster_info

We could think of these as the "Unexpected Action", "Happy Films", "Cartoon Adventure", and "Sinister Drama" clusters. Perhaps we could use this information to develop four new films that would cover a wide variety of tastes. 

## Movie-level Clustering
Now that we've covered some ground regarding how Kmeans clusters users based on their genre tastes, let's take a bigger bite and look at how users rated individual movies. To do that, we'll shape the dataset in the form of userId vs user rating for each movie. For example, let's look at a subset of the dataset:

In [ ]:
# Merge the two tables then pivot so we have Users X Movies dataframe
ratings_title = pd.merge(df_ratings, df_movies[['movieId', 'title']], on='movieId')
df_user_movie_ratings = pd.pivot_table(ratings_title, index='userId', columns= 'title', values='rating')

print('dataset dimensions: ', df_user_movie_ratings.shape, '\n\nSubset example:')
df_user_movie_ratings.iloc[10:20, :10]

Most users have not rated and watched most movies. The dominance of NaN values can be an important issue. Can you tell why?

Same as with text, datasets like this are called "sparse" because only a small number of cells have values. To get around this, let's sort by the most rated movies, and the users who have rated the most number of movies. That will present a more 'dense' region when we peak at the top of the dataset.

In [ ]:
n_movies = 30
n_users = 20
top_movies = df_user_movie_ratings.count().sort_values()[-n_movies:].index[::-1]
top_users = df_user_movie_ratings.count(axis=1).sort_values()[-n_users:].index[::-1]
top_mat = df_user_movie_ratings.loc[top_users, top_movies]
plt.figure(figsize = (15,8))
shorter_titles = [c[:30] for c in top_mat.columns]
g = sns.heatmap(top_mat, cmap=plt.cm.coolwarm_r, xticklabels=shorter_titles)
g.xaxis.set_ticks_position('top')
plt.setp(g.get_xticklabels(), rotation=90)
plt.xlabel('')
g.set_facecolor('black')
plt.show()

Each column is a movie. Each row is a user. The color of the cell is how the user rated that movie based on the scale on the right of the graph.

Notice how some cells are black? This means the respective user did not rate that movie. This is an issue you'll come across when clustering in real life. Unlike the clean example we started with, real-world datasets can often be sparse and not have a value in each cell of the dataset. This makes it less straightforward to cluster users directly by their movie ratings as k-means generally does not like missing values. Can you think what to do about it?

In [ ]:
# We will keep the top 1000 movies to cluster people
n_movies = 1000
top_movieId = ratings_title.movieId.value_counts()[:1000].index
ratings_top_movies = ratings_title[np.in1d(ratings_title.movieId, top_movieId)]
# Create a sparse version of the Users X Movies dataframe. This will impute a value of 0 for missing data.
titles_c = np.array(sorted(ratings_top_movies.title.unique()))
users_c = np.array(sorted(ratings_top_movies.userId.unique()))
row = pd.Categorical(ratings_top_movies.userId, categories=users_c, ordered=True).codes
col = pd.Categorical(ratings_top_movies.title, categories=titles_c, ordered=True).codes
sparse_ratings = csr_matrix((ratings_top_movies.rating, (row, col)), shape=(users_c.size, titles_c.size))
sparse_ratings

Let's cluster by movies and take a look at the results! 

In [ ]:
k_clusters = 20
kmeans = KMeans(n_clusters=k_clusters, random_state=42)
clusters = kmeans.fit_predict(sparse_ratings).astype(int)

data = defaultdict(list)
top = 5
for cluster in range(k_clusters):
    top_ixs = np.asarray(sparse_ratings[clusters == cluster, :].mean(axis=0).argsort()[0, -top:]).reshape(-1)
    for i, ix in enumerate(top_ixs):
        data["Top {0} Movie".format(i + 1)].append(titles_c[ix])
    counts = clusters[clusters == cluster].size
    data["count"].append(counts)
    
cols = ['count'] + ["Top {0} Movie".format(i+1) for i in range(top)]
cluster_info = pd.DataFrame(data)[cols].transpose()
cluster_info.columns = ["cluster {0}".format(i) for i in range(k_clusters)]
cluster_info.iloc[:, cluster_info.loc['count'].argsort()[::-1]]

We can see some clear trends in the clusters. For example: 
* Cluster 18 likes crime movies
* Clusters 4 and 1 like Star Wars and Terminator.
* Cluster 10 likes Lord of the Rings.

Can you think of other ways to represent users and cluster them?

## Nearest Neighbors: Recommending Movies

Imagine that we want to make movie recommendations to a user. Can we use similarity to do this? As an example, let's find the nearest neighbors to userId 1.

In [25]:
# Find k=10 nearest neighbors
k = 10
nn_model = NearestNeighbors(n_neighbors=k+1).fit(sparse_ratings)
distances, indices = nn_model.kneighbors(sparse_ratings[0,:])

Now, let's find the movies that were liked the most by taking the average rating. Do you think this is a good idea? Why yes or why not?

In [ ]:
# Get average ratings for each movie according to nearest neighbors
avg_ratings = np.asarray(sparse_ratings[indices[0, 1:], :].mean(axis=0)).reshape(-1)
# Get top recommendations
k_rec = 10
top_recommendations = titles_c[avg_ratings.argsort()[::-1][:k_rec]]
# Check  whether the user has already seen these movies.
for title in top_recommendations:
    user_rating = df_user_movie_ratings.loc[1, title]
    if  pd.isnull(user_rating):
        print("RECOMMEND:", title)
    else:
        print("USER GAVE RATING", user_rating, "TO",  title)

The nearest neighbors like Terminator 2. What if we wanted to predict how much the user would like this recommendation? How could we do that?

That's a supervised learning task. If we have a target variable to estimate/predict and labels for a training set,  we can do prediction directly using similarity.

In this case, our label would be the rating for the Terminator 2 movie. One way to use similarity to build a predictor<sup>&dagger;</sup> is to use a **Nearest Neighbor algorithm**.  The idea is: to predict the value of the target variable for a data item, first find the most similar (closest) training data items.  The **k-Nearest-Neighbor** or **kNN** algorithm chooses the closest `k` data points.  Then, gather the values of the target variable for them, and then combine them somehow.  So, to classify, one might combine them by having them vote their classes. (How would you combine to compute probability estimates? What about a regression problem?)

For now, let's take a look at the ratings of the top recommended movies for each nearest neighbor.

<sup>&dagger;</sup>There's an interesting question as to whether we're actually building a *model* here.

In [ ]:
top_neighbors_movies = df_user_movie_ratings.iloc[indices[0, 1:]][top_recommendations] 
top_neighbors_movies

Let's take the average rating now (excluding people that did not rate the movie) and compare that to the user ratings.

In [ ]:
df_rec = pd.concat([top_neighbors_movies.mean(), df_user_movie_ratings.loc[1, top_recommendations]], axis=1)
df_rec.columns = ['Avg. Rating', 'UserId 1 Ratings']
df_rec

Seems like Terminator might not be the best recommendation after all! But how come it was listed as the top recommendation? What would you do differently?

### Finding similar movies

Suppose we recommend 'Aliens' to userId 1, and the user loves the movie. He asks us to recommend a movie similar to that one. How would you do that? HINT: It's pretty much the same thing we already did.

In [ ]:
movie = "Aliens (1986)"
movie_ix = np.where(movie == titles_c)[0][0]
# Find k=10 nearest movies
k = 10
nn_model = NearestNeighbors(n_neighbors=k+1).fit(sparse_ratings.T)
distances, indices = nn_model.kneighbors(sparse_ratings.T[movie_ix,:])
titles_c[indices[0]]

### Prediction via similarity

In a previous example we decided to predict ratings based on the 10 nearest neighbors. But why not just the nearest neighbor? Or why not the 100 nearest neighbors? To illustrate this, let's go back to our earlier example of describing users in terms of the genres they like. 

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
all_genres = df_normalized_genre.columns[df_normalized_genre.columns != 'cluster']
genres = ['Drama', 'Fantasy']
df_normalized_genre['cluster'] = kmeans.fit_predict(df_normalized_genre[all_genres]).astype(int)
sns.lmplot(genres[0], genres[1], data=df_normalized_genre, hue='cluster', fit_reg=False, height=8)
plt.show()

Suppose we want to find to which cluster each user belongs based only on their taste for fantasy and drama. How would the decision surface look like?

In [ ]:
# Let's start by slitting the data
from sklearn.model_selection import train_test_split
X = df_normalized_genre[genres]
Y = df_normalized_genre['cluster']

for i, k in enumerate([100, 10, 1]):
    plt.figure(figsize=[13,10])
    model = KNeighborsClassifier(n_neighbors=k)
    Decision_Surface(X, genres[0], genres[1], Y, model)
    plt.title("K=" + str(k))
    plt.show()

Does this look familiar? How should we determine which K is the best one? You guessed it, in the same way we find optimal complexity control parameters.

In [ ]:
for k in [1, 10, 20, 50, 100, 200, 400]:
    model = KNeighborsClassifier(n_neighbors=k)
    auc = cross_val_score(model, X, Y, scoring="roc_auc", cv=10).mean()
    print("AUC: {0} with K {1}".format(round(auc*100, 2), k))